### Student Activity: Portfolio Planner Part I

This program is part 1 of a two-part student activity and begins the following:

`PART 1: Portfolio Optimization via Risk Evaluation`

Reads in the CSV datasets of 10 stocks, calculates the volatility of each stock, drops the top-five highly volatile stocks, sets allocations for the remaining stocks based on risk/volatility, and calculates the returns of a hypothetical `$10,000` investment for the constructed portfolio.

`PART 2: Portfolio Optimization via Correlation and Return-to-Risk (Sharpe Ratio) Evaluations`

To be continued in the second part!


In [1]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
from pathlib import Path
%matplotlib inline

### Read CSVs as DataFrames 

In [2]:
# Set the file paths
bk_data = Path("../Resources/bk_data.csv")
fang_data = Path("../Resources/fang_data.csv")
jnj_data = Path("../Resources/jnj_data.csv")
luv_data = Path("../Resources/luv_data.csv")
mu_data = Path("../Resources/mu_data.csv")
nke_data = Path("../Resources/nke_data.csv")
sbux_data = Path("../Resources/sbux_data.csv")
t_data = Path("../Resources/t_data.csv")
wdc_data = Path("../Resources/wdc_data.csv")
wrk_data = Path("../Resources/wrk_data.csv")

# Read the CSVs and set the `date` column as a datetime index to the DataFrame
bk_df = pd.read_csv(bk_data, index_col="date", infer_datetime_format=True, parse_dates=True)
fang_df = pd.read_csv(fang_data, index_col="date", infer_datetime_format=True, parse_dates=True)
jnj_df = pd.read_csv(jnj_data, index_col="date", infer_datetime_format=True, parse_dates=True)
luv_df = pd.read_csv(luv_data, index_col="date", infer_datetime_format=True, parse_dates=True)
mu_df = pd.read_csv(mu_data, index_col="date", infer_datetime_format=True, parse_dates=True)
nke_df = pd.read_csv(nke_data, index_col="date", infer_datetime_format=True, parse_dates=True)
sbux_df = pd.read_csv(sbux_data, index_col="date", infer_datetime_format=True, parse_dates=True)
t_df = pd.read_csv(t_data, index_col="date", infer_datetime_format=True, parse_dates=True)
wdc_df = pd.read_csv(wdc_data, index_col="date", infer_datetime_format=True, parse_dates=True)
wrk_df = pd.read_csv(wrk_data, index_col="date", infer_datetime_format=True, parse_dates=True)

# Display a few rows
wrk_df.head()

/var/folders/r4/jh2rbs2n3js2641fckxtvh_c0000gn/T/ipykernel_60765/3262590508.py:14: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  bk_df = pd.read_csv(bk_data, index_col="date", infer_datetime_format=True, parse_dates=True)
/var/folders/r4/jh2rbs2n3js2641fckxtvh_c0000gn/T/ipykernel_60765/3262590508.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  bk_df = pd.read_csv(bk_data, index_col="date", infer_datetime_format=True, parse_dates=True)
/var/folders/r4/jh2rbs2n3js2641fckxtvh_c0000gn/T/ipykernel_60765/3262590508.py:15: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A str

,close
date,
2019-05-20,35.15
2019-05-17,36.66
2019-05-16,37.42
2019-05-15,37.44
2019-05-14,37.19


### Combine DataFrames, Sort Index, and Rename Columns

In [ ]:
# Create a new pivot table where the columns are the closing prices for each ticker


# Sort datetime index in ascending order (past to present)


# Set column names


# Display a few rows


### Calculate Daily Returns

In [ ]:
# Use the `pct_change` function to calculate daily returns


### Evaluate Riskiness of Stocks

In [ ]:
# Use the `std` function and multiply by the square root of the number of trading days in a year to get annualized volatility


### Drop High Volatility Stocks

In [ ]:
# Drop the five stocks with the highest volatility in daily returns


### Set Portfolio Allocations/Weights and Calculate Portfolio Daily Returns

In [ ]:
# Set weights for corresponding risk profile of stocks, use the `dot` function to multiply each weight by the corresponding stock daily return
# BK, LUV, NKE, SBUX, T


### Calculate Cumulative Returns

In [ ]:
# Use the `cumprod` function to calculate cumulative returns


### Plot Return of Portfolio Starting with Initial Investment of `$10,000`

In [ ]:
# Plot the returns of the portfolio in terms of money
